In [ ]:
 !pip install tables==3.6.1
 !pip install socceraction==0.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ast import literal_eval
import warnings

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import brier_score_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

import socceraction.vaep.features as features
import socceraction.vaep.labels as labels
from socceraction.spadl.wyscout import convert_to_spadl
from socceraction.vaep.formula import value
from tqdm.notebook import tqdm
from xgboost import XGBClassifier

In [ ]:
import warnings
from io import BytesIO
from pathlib import Path
from urllib.parse import urlparse
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile, is_zipfile

import pandas as pd
import socceraction.vaep.features as features
import socceraction.vaep.labels as labels
from sklearn.metrics import brier_score_loss, roc_auc_score
from socceraction.spadl.wyscout import convert_to_spadl
from socceraction.vaep.formula import value
from tqdm.notebook import tqdm
from xgboost import XGBClassifier

In [ ]:
warnings.filterwarnings('ignore', category=pd.io.pytables.PerformanceWarning)

In [ ]:
# This section downloads the Wyscout dataset, collects the required information about the match events, and converts the match events into the SPADL representation.

# Download the Wyscout dataset; (Not needed in Kaggle)
# Construct an HDF5 file named wyscout.h5 that contains the relevant information from the dataset;
# Convert the wyscout.h5 file into a spadl.h5 file that contains the same information in the SPADL representation.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_files = {
    'events': 'https://ndownloader.figshare.com/files/14464685',  # ZIP file containing one JSON file for each competition
    'matches': 'https://ndownloader.figshare.com/files/14464622',  # ZIP file containing one JSON file for each competition
    'players': 'https://ndownloader.figshare.com/files/15073721',  # JSON file
    'teams': 'https://ndownloader.figshare.com/files/15073697'  # JSON file
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%mkdir -p '/content/drive/My Drive/football/'
%cd '/content/drive/My Drive/football/'

In [ ]:
for url in tqdm(data_files.values()):
    url_s3 = urlopen(url).geturl()
    path = Path(urlparse(url_s3).path)
    file_name = path.name
    file_local, _ = urlretrieve(url_s3, file_name)
    if is_zipfile(file_local):
        with ZipFile(file_local) as zip_file:
            zip_file.extractall()

In [ ]:
def read_json_file(filename):
    with open(filename, 'rb') as json_file:
        return BytesIO(json_file.read()).getvalue().decode('unicode_escape')

**in a HDF5 file, what we call "directories" or "folders" on our computers, are called groups and what we call files on our computer are called datasets.**

**TEAMS**


In [ ]:
json_teams = read_json_file('teams.json')
df_teams = pd.read_json(json_teams)

In [ ]:
df_teams.head(5)

In [ ]:
 df_teams.to_hdf('wyscout.h5', key='teams', mode='w')

**PLAYERS**

 read the players.json file into a DataFrame object and store that object in the wyscout.h5 HDF5 file under the key players.

In [ ]:
json_players = read_json_file('players.json')
df_players = pd.read_json(json_players)

In [ ]:
df_players.head(2)

In [ ]:
df_players.to_hdf('wyscout.h5', key='players', mode='a')

**Matches**

Uncomment the competitions that you want to include in your dataset.

In [ ]:
competitions = [
     'England',
     'France',
     'Germany',
     'Italy',
    'Spain',
#   'European Championship',
#   'World Cup'
]

read the matches.json files for the selected competitions into a DataFrame object and store that object in the wyscout.h5 HDF5 file under the key matches.

In [ ]:
dfs_matches = []
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_matches = f'matches_{competition_name}.json'
    json_matches = read_json_file(file_matches)
    df_matches = pd.read_json(json_matches)
    dfs_matches.append(df_matches)
df_matches = pd.concat(dfs_matches)

In [ ]:
df_matches.head(3)

In [ ]:
df_matches.to_hdf('wyscout.h5', key='matches', mode='a')

**EVENTS**

read the events.json files for the selected competitions into a DataFrame object and store that object in the wyscout.h5 HDF5 file under the key events/match_match-id

In [ ]:
for competition in competitions:
    competition_name = competition.replace(' ', '_')
    file_events = f'events_{competition_name}.json'
    json_events = read_json_file(file_events)
    df_events = pd.read_json(json_events)
    df_events_matches = df_events.groupby('matchId', as_index=False)
    for match_id, df_events_match in df_events_matches:
        df_events_match.to_hdf('wyscout.h5', key=f'events/match_{match_id}', mode='a')

**Convert the Wyscout data to the SPADL representation**

The following cell calls the convert_to_spadl function from the socceraction library to convert the wyscout.h5 HDF5 file into the spadl.h5 HDF5 file.

In [ ]:
convert_to_spadl('wyscout.h5', 'spadl.h5')

**Value game states**

1-Generate the features to describe the game states;

2-Generate the labels that capture the value of the game states;

3-Compose a dataset by selecting a set of features and the labels of the game states;

4-Train predictive machine learning models using the dataset;

5-Value the game states using the trained predictive machine learning model.

In [ ]:
df_games = pd.read_hdf('spadl.h5', key='games')
df_actiontypes = pd.read_hdf('spadl.h5', key='actiontypes')
df_bodyparts = pd.read_hdf('spadl.h5', key='bodyparts')
df_results = pd.read_hdf('spadl.h5', key='results')

In [ ]:
nb_prev_actions = 3

**Generate game state features**

The following cell lists a number of feature generators from the features module in the socceraction library. Each function expects either a DataFrame object containing actions (i.e., individual actions) or a list of DataFrame objects containing consecutive actions (i.e., game states), and returns the corresponding feature for the individual action or game state.

In [ ]:
functions_features = [
    features.actiontype_onehot,
    features.bodypart_onehot,
    features.result_onehot,
    features.goalscore,
    features.startlocation,
    features.endlocation,
    features.movement,
    features.space_delta,
    features.startpolar,
    features.endpolar,
    features.team,
    features.time_delta
]

The following cell generates game states from consecutive actions in each game and computes the features for each game state.

1- Obtain the actions for the game (i.e., df_actions) by looping through the games;

2- Construct game states of a given length from the actions (i.e., dfs_gamestates);

3- Compute the features for the constructed game states (i.e., df_features) by looping through the list of feature generators.

In [ ]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    dfs_gamestates = features.gamestates(df_actions, nb_prev_actions=nb_prev_actions)
    dfs_gamestates = features.play_left_to_right(dfs_gamestates, game['home_team_id'])
    
    df_features = pd.concat([function(dfs_gamestates) for function in functions_features], axis=1)
    df_features.to_hdf('features.h5', key=f'game_{game_id}')

  0%|          | 0/1826 [00:00<?, ?it/s]

**Generate game state labels**

The following cell lists a number of label generators from the labels module in the socceraction library. Each function expects either a DataFrame object containing actions (i.e., individual actions) or a list of DataFrame objects containing consecutive actions (i.e., game states), and returns the corresponding label for the individual action or game state.

In [ ]:
functions_labels = [
    labels.scores,
    labels.concedes
]

The following cell computes the labels for each action:

1- Obtain the actions for the game (i.e., df_actions) by looping through the games;

2- Compute the labels for the actions (i.e., df_labels) by looping through the list of label generators.

In [ ]:
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .reset_index(drop=True)
    )
    
    df_labels = pd.concat([function(df_actions) for function in functions_labels], axis=1)
    df_labels.to_hdf('labels.h5', key=f'game_{game_id}')

**Generate dataset**

The following cell generates a list of names for the features to be included in the dataset.

In [ ]:
columns_features = features.feature_column_names(functions_features, nb_prev_actions=nb_prev_actions)

The following cell obtains the relevant features for each game and stores them in the df_features DataFrame object.

In [ ]:
dfs_features = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_features = pd.read_hdf('features.h5', key=f'game_{game_id}')
    dfs_features.append(df_features[columns_features])
df_features = pd.concat(dfs_features).reset_index(drop=True)

In [ ]:
df_features.head(5)

The following cell lists the names of the labels to be included in the dataset.


In [ ]:
columns_labels = [
    'scores',
    'concedes'
]

The following cell obtains the relevant labels for each game and stores them in the df_labels DataFrame object.

In [ ]:
dfs_labels = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_labels = pd.read_hdf('labels.h5', key=f'game_{game_id}')
    dfs_labels.append(df_labels[columns_labels])
df_labels = pd.concat(dfs_labels).reset_index(drop=True)

In [ ]:
df_labels.head(5)

,scores,concedes
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False


**Train classifiers**

The following cell trains an XGBoost classifier for each label using the computed features. For each label:

1- Construct an XGBoost classifier with default hyperparameters;

2- Train the classifier using the computed features and the label;

3- Store the trained classifier in the models dict.

In [ ]:
%%time
models = {}
for column_labels in columns_labels:
    model = XGBClassifier(
        eval_metric='logloss',
        use_label_encoder=False
    )
    model.fit(df_features, df_labels[column_labels])
    models[column_labels] = model

**Estimate probabilities**

The following cell predicts the labels for the game states using the trained XGBoost classifier. For each label:

1- Retrieve the model for the label;

2- Estimate the probabilities of the labels being False and True given the computed features;

3- Keep the probabilities for the True label;

4- Store the probabilities as a Series object in the dfs_predictions dict.

In [ ]:
dfs_predictions = {}
for column_labels in columns_labels:
    model = models[column_labels]
    probabilities = model.predict_proba(df_features)
    predictions = probabilities[:, 1]
    dfs_predictions[column_labels] = pd.Series(predictions)
df_predictions = pd.concat(dfs_predictions, axis=1)

In [ ]:
df_predictions.head(5)

The following cell obtains the game_id for each action in order to store the predictions per game.

In [ ]:
dfs_game_ids = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    dfs_game_ids.append(df_actions['game_id'])
df_game_ids = pd.concat(dfs_game_ids, axis=0).astype('int').reset_index(drop=True)

The following cell concatenates the DataFrame objects with predictions and game_ids for each action into a single DataFrame object.

In [ ]:
df_predictions = pd.concat([df_predictions, df_game_ids], axis=1)

In [ ]:
df_predictions.head(5)

The following cell groups the predictions per game based on their game_id.

In [ ]:
df_predictions_per_game = df_predictions.groupby('game_id')

The following cell stores the predictions in the predictions.h5 HDF5 file per game.

In [ ]:
for game_id, df_predictions in tqdm(df_predictions_per_game):
    df_predictions = df_predictions.reset_index(drop=True)
    df_predictions[columns_labels].to_hdf('predictions.h5', key=f'game_{game_id}')

**Value on-the-ball actions**

Note: The code in this section is based on the 4-compute-vaep-values.ipynb notebook in the socceraction repository.

In [ ]:
df_players = pd.read_hdf('spadl.h5', key='players')
df_teams = pd.read_hdf('spadl.h5', key='teams')

In [ ]:
dfs_values = []
for _, game in tqdm(df_games.iterrows(), total=len(df_games)):
    game_id = game['game_id']
    df_actions = pd.read_hdf('spadl.h5', key=f'actions/game_{game_id}')
    df_actions = (df_actions
        .merge(df_actiontypes, how='left')
        .merge(df_results, how='left')
        .merge(df_bodyparts, how='left')
        .merge(df_players, how='left')
        .merge(df_teams, how='left')
        .reset_index(drop=True)
    )
    
    df_predictions = pd.read_hdf('predictions.h5', key=f'game_{game_id}')
    df_values = value(df_actions, df_predictions['scores'], df_predictions['concedes'])
    
    df_all = pd.concat([df_actions, df_predictions, df_values], axis=1)
    dfs_values.append(df_all)

In [ ]:
df_values = (pd.concat(dfs_values)
    .sort_values(['game_id', 'period_id', 'time_seconds'])
    .reset_index(drop=True)
)

In [ ]:
df_values[
    ['short_name', 'scores', 'concedes', 'offensive_value', 'defensive_value', 'vaep_value']
].head(10)

**Rate players**

Note: The code in this section is based on the 5-top-players.ipynb notebook in the socceraction repository.

**Rate according to total VAEP value**

In [ ]:
df_ranking = (df_values[['player_id', 'team_name', 'short_name', 'vaep_value']]
    .groupby(['player_id', 'team_name', 'short_name'])
    .agg(vaep_count=('vaep_value', 'count'), vaep_sum=('vaep_value', 'sum'))
    .sort_values('vaep_sum', ascending=False)
    .reset_index()
)

In [ ]:
df_ranking.head(10)

**Rate according to total VAEP value per 90 minutes**

In [ ]:
df_player_games = pd.read_hdf('spadl.h5', 'player_games')
df_player_games = df_player_games[df_player_games['game_id'].isin(df_games['game_id'])]

In [ ]:
df_minutes_played = (df_player_games[['player_id', 'minutes_played']]
    .groupby('player_id')
    .sum()
    .reset_index()
)

In [ ]:
df_minutes_played.head(5)

In [ ]:
df_ranking_p90 = df_ranking.merge(df_minutes_played)
df_ranking_p90 = df_ranking_p90[df_ranking_p90['minutes_played'] > 360]
df_ranking_p90['vaep_rating'] = df_ranking_p90['vaep_sum'] * 90 / df_ranking_p90['minutes_played']
df_ranking_p90 = df_ranking_p90.sort_values('vaep_rating', ascending=False)

In [ ]:
df_ranking_p90.head(5)

In [ ]:
df_ranking_p90.to_csv('ranking.csv', index=False)